In [1]:
#importing what we need
import pandas as pd 
import re #regex

%store -r 

In [2]:
# read in data

years = ["2020", "2019", "2018", "2017"] #if we get new data, we can just add that here
dataframes = []


for year in years: 
    dataframe = pd.read_excel('FINAL ' + year + ' RAA Data.xlsx')
    dataframes.append(dataframe)
    

In [3]:
dataframes[-1]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ID,Priority,CallType,ChiefComplaint,Impression,Procedures,Medications,Vitals,Narrative,Outcome
0,0,0,0,2017_2,1,Abdominal Pain,{Abdominal Pain - Generalized},{GI/GU - Abdominal Generalized (R10.84)}{Weakn...,{22:34:00: Cardiac Monitor (46825001)}{22:35:1...,{22:44:00: Ondansetron (Zofran) (26225)},{22:34:37: Pulse-94 Resp-12 BP-166/84 GCS-15 G...,D- RAA responded immediately to 3246 BROAD ROC...,"Treated, Transported by EMS"
1,1,1,1,2017_3,1,Abdominal Pain,{Abdominal Pain - Generalized}{Atraumatic Blee...,{GI/GU - Abdominal Pain Acute Onset (R10.0)}{G...,{16:27:00: Cardiac Monitor (46825001)}{16:36:0...,{16:42:42: Fentanyl (4337)},{16:29:35: Pulse-102 Resp-22 BP-126/81 GCS-15 ...,A: Dispatched for a female complaining of abdo...,"Treated, Transported by EMS"
2,2,2,2,2017_4,1,Abdominal Pain,{Abdominal Pain - Generalized}{General Illness...,{GI/GU - Abdominal Generalized (R10.84)}{GI/GU...,{21:34:22: Cardiac Monitor (46825001)}{21:36:0...,{21:40:25: Ondansetron (Zofran) (26225)},{21:33:56: Pulse-85 Resp-20 BP-146/92 GCS-15 G...,D: Unit 576 dispatched priority 1 to an apartm...,"Treated, Transported by EMS"
3,3,3,3,2017_5,1,Abdominal Pain,{Abdominal Pain - Pancreatitis},{GI/GU - Abdominal Generalized (R10.84)}{GI/GU...,{07:45:05: 12-Lead ECG (268400002)}{07:46:31: ...,{07:52:00: Normal Saline (125464)},{07:45:05: Pulse-46 Resp-18 BP-108/66 GCS-15 G...,"D: Dispatched for a sick person, that was cha...","Treated, Transported by EMS"
4,4,4,4,2017_6,1,Abdominal Pain,{Abdominal Pain - Generalized},{GI/GU - Nausea (With Vomiting) (R11.2)}{Weakn...,{08:48:00: Cardiac Monitor (46825001)}{08:50:0...,{08:53:00: Ondansetron (Zofran) (26225)},{08:48:03: Pulse-95 Resp-14 BP-116/79 GCS-15 G...,D- RAA responded immediately to 3208 NINE MILE...,"Treated, Transported by EMS"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7297,7297,7297,7297,2017_7299,2,Unknown Problem,{Pain - Chest (Non-Cardiac)},{Pain - Chest (presumed non-cardiac) (R07.89)}...,{08:20:00: Cardiac Monitor (46825001)}{08:27:5...,{08:25:24: Aspirin (1191)},{08:18:00: Pulse-98 Resp-12 BP-157/98 GCS-15 G...,D: EMS 480 dispatched priority 2 for the Medic...,"Treated, Transported by EMS"
7298,7298,7298,7298,2017_7300,2,Unknown Problem,{Pain - Hip},{Fever (R50.9)}{Pain - Chest (presumed non-car...,{16:09:00: Cardiac Monitor (46825001)}{16:09:4...,{16:07:00: Aspirin (1191)}{16:15:00: Ondansetr...,{16:07:14: Pulse-124 Resp-16 BP-143/99 GCS-15 ...,D - Medic 477 was dispatched for a medical ala...,"Treated, Transported by EMS"
7299,7299,7299,7299,2017_7301,2,Unknown Problem,{Altered Consciousness - Disoriented - Diabeti...,{Endocrine - Hypoglycemia - Diabetic (E13.64)}...,{14:24:00: Cardiac Monitor (46825001)}{14:25:0...,{14:26:00: Dextrose 10% (237648)},{14:24:05: Pulse-91 Resp-20 BP-216/140 GCS-9 G...,Dispatched: Unknown Problem Units Responding...,"Treated, Transported by EMS"
7300,7300,7300,7300,2017_7302,3,Unknown Problem,{Abdominal Pain - Generalized}{General Illness...,{GI/GU - Abdominal Generalized (R10.84)}{GI/GU...,{01:09:30: Cardiac Monitor (46825001)}{01:11:5...,{01:15:54: Ondansetron (Zofran) (26225)},{01:07:48: Pulse-80 Resp-16 BP-199/93 GCS-15 G...,D: Unit 587 dispatched priority 1 (downgraded ...,"Treated, Transported by EMS"


In [4]:
#define dictionaries
impression_id_dictionaries = [] 
impression_num_dictionaries = []

pro_id_dictionaries = []
pro_num_dictionaries = []

med_id_dictionaries = []
med_num_dictionaries = []

for i in range(len(years)):
    impression_id_dictionaries.append({})
    impression_num_dictionaries.append({})
    
    pro_id_dictionaries.append({})
    pro_num_dictionaries.append({})
    
    med_id_dictionaries.append({})
    med_num_dictionaries.append({})
    

In [5]:
#for each year, go through the impression column
#for each column entry, remove front and last characters { and }
#then parse by }{ 
# isolate unique id and add to dictionary

In [6]:
# filling keys for impression dictionaries

for i in range(len(year)):
    df = dataframes[i]
    id_dictionary = impression_id_dictionaries[i]
    num_dictionary = impression_num_dictionaries[i]
    
    df['Impression'] = df['Impression'].str[1:-1] #remove {} 
    impressions = df['Impression'].str.split('}{')
    
    for row in range(len(impressions)):
        try: #for NaN values
            call_id = df['ID'][row]
            for each in impressions[row]:
                id_start, id_end = re.search(r'[A-Z][0-9]', each).span()
                icd = each[id_start:-1]
                if not icd in id_dictionary:
                    id_dictionary[icd] = []
                    num_dictionary[icd] = 0
                id_dictionary[icd].append(call_id)
                num_dictionary[icd] += 1
        except TypeError:
            pass #skip and keep going if nan
        


In [7]:
# filling keys for procedure dictionaries

for i in range(len(year)):
    df = dataframes[i]
    id_dictionary = pro_id_dictionaries[i]
    num_dictionary = pro_num_dictionaries[i]
    
    df['Procedures'] = df['Procedures'].str[1:-1] #remove {}
    procedures = df['Procedures'].str.split('}{')
    
    for row in range(len(procedures)):
        try:
            call_id = df['ID'][row]
            for each in procedures[row]:
                try:
                    id_start, id_end = re.search(r'\([0-9]{2,}', each).span()
                    snomed = each[id_start+1:id_end]
                    if not snomed in id_dictionary:
                        id_dictionary[snomed] = []
                        num_dictionary[snomed] = 0
                    id_dictionary[snomed].append(call_id)
                    num_dictionary[snomed] += 1
                except AttributeError:
                    pass
        except TypeError:
            pass #skip and keep going if nan

        


In [8]:
# filling keys for medication dictionaries

for i in range(len(year)):
    df = dataframes[i]
    id_dictionary = med_id_dictionaries[i]
    num_dictionary = med_num_dictionaries[i]
    
    df['Medications'] = df['Medications'].str[1:-1] #remove {} 
    meds = df['Medications'].str.split('}{')
    
    for row in range(len(meds)):
        try: #for NaN values
            call_id = df['ID'][row]
            for each in meds[row]:
                end = each.rfind('(') - 1
                med_id = each[end+2:-1]
                if not med_id in id_dictionary:
                    id_dictionary[med_id] = []
                    num_dictionary[med_id] = 0
                id_dictionary[med_id].append(call_id)
                num_dictionary[med_id] += 1
        except TypeError:
            pass #skip and keep going if nan
        

In [9]:
#check impression dictionaries

for i in range(len(years)):
    print(years[i], 'impressions:')
    print(impression_id_dictionaries[i])
    print('\n')


2020 impressions:
{'G89.2': ['2020_2', '2020_112', '2020_178', '2020_180', '2020_490', '2020_636', '2020_645', '2020_886', '2020_886', '2020_981', '2020_981', '2020_1267', '2020_1673', '2020_1695', '2020_1804', '2020_1864', '2020_1930', '2020_2216', '2020_2233', '2020_2439', '2020_2491', '2020_2674', '2020_3016', '2020_3016', '2020_3409', '2020_3409', '2020_4090', '2020_4090', '2020_4251', '2020_4325', '2020_4352', '2020_4497', '2020_4497', '2020_4732', '2020_5104', '2020_5204', '2020_5304', '2020_5813', '2020_5866', '2020_5871', '2020_5883', '2020_6031', '2020_6031', '2020_6051', '2020_6135', '2020_6202', '2020_6205', '2020_6295', '2020_6595', '2020_6862', '2020_6934', '2020_7159', '2020_7159', '2020_7167', '2020_7651', '2020_7782', '2020_8145', '2020_8226', '2020_8261', '2020_8486', '2020_8813', '2020_8853', '2020_9278', '2020_9278', '2020_9484', '2020_9484', '2020_9812', '2020_9852', '2020_9872', '2020_10733', '2020_10902', '2020_11432', '2020_11485', '2020_11983', '2020_12016', '20

In [10]:
#check procedure dictionaries

for i in range(len(years)):
    print(years[i], 'procedures:')
    print(pro_num_dictionaries[i])
    print('\n')

2020 procedures:
{'428803005': 14767, '268400002': 12010, '386423001': 158, '392230005': 9248, '49689007': 285, '353008': 144, '230040009': 196, '425543005': 1005, '425447009': 825, '385857005': 29, '429283006': 238, '232674004': 144, '430824005': 310, '89666000': 234, '302789003': 121, '47545007': 195, '398041008': 127, '56251003': 501, '424287005': 6, '79321009': 58, '450611000124100': 160, '405427009': 173, '302488007': 6, '7443007': 91, '182692007': 423, '26906007': 41, '406164000': 17, '426220008': 189, '422618004': 290, '236973005': 2, '51717002': 95, '243140006': 105, '449199004': 32, '182531007': 43, '233169004': 16, '235425002': 22, '426498007': 78, '229585002': 26, '409088002': 5, '425519007': 10, '271280005': 4, '447686008': 79, '20655006': 32, '19861002': 6, '128968000': 25, '182705007': 17, '18590009': 8, '46825001': 9, '225116006': 4, '250980009': 11, '398074008': 5, '225358003': 23, '398279005': 3, '225150002': 10, '58715004': 3, '104690002': 10, '133901003': 3, '2326850

In [11]:
#check medication dictionaries

for i in range(len(years)):
    print(years[i], 'medications:')
    print(med_id_dictionaries[i])
    print('\n')

2020 medications:
{'7806': ['2020_5', '2020_15', '2020_17', '2020_18', '2020_26', '2020_34', '2020_48', '2020_53', '2020_54', '2020_64', '2020_72', '2020_75', '2020_76', '2020_77', '2020_85', '2020_89', '2020_89', '2020_91', '2020_95', '2020_100', '2020_101', '2020_102', '2020_102', '2020_104', '2020_115', '2020_122', '2020_122', '2020_132', '2020_140', '2020_140', '2020_141', '2020_150', '2020_154', '2020_155', '2020_155', '2020_156', '2020_156', '2020_161', '2020_162', '2020_164', '2020_172', '2020_176', '2020_180', '2020_181', '2020_182', '2020_183', '2020_189', '2020_193', '2020_196', '2020_200', '2020_206', '2020_208', '2020_212', '2020_219', '2020_219', '2020_221', '2020_222', '2020_224', '2020_230', '2020_231', '2020_232', '2020_233', '2020_233', '2020_233', '2020_235', '2020_241', '2020_245', '2020_247', '2020_247', '2020_257', '2020_257', '2020_262', '2020_263', '2020_268', '2020_269', '2020_277', '2020_280', '2020_283', '2020_283', '2020_294', '2020_295', '2020_296', '2020_29

In [12]:
# convert dictionaries to dataframes for easier views

#NUMBER DICTIONARIES
impression_num_df = []
intervention_num_df = []

#convert numbers dictionaries into tables
for i in range(len(years)):
    imp_df = pd.DataFrame(list(impression_num_dictionaries[i].items()),columns = ['ICD-10-CM','Frequency (Number of Calls)']) #convert
    imp_df.style.set_caption('Frequncy of Impressions for ' + years[i] + ' RAA Data')

    imp_df = imp_df.join(grouped_imp_df.set_index('ICD-10-CM'), on='ICD-10-CM')
    
    impression_num_df.append(imp_df)

    
    pro_df = pd.DataFrame(list(pro_num_dictionaries[i].items()),columns = ['Snomed', 'Proced. Frequency (Number of Calls)']) #convert
    med_df = pd.DataFrame(list(med_num_dictionaries[i].items()),columns = ['RxCui', 'Med. Frequency (Number of Calls)']) #convert


    pro_df = pro_df.join(grouped_pro_df.astype({'Snomed': 'str'}).set_index('Snomed'), on='Snomed')
    med_df = med_df.join(grouped_med_df.astype({'RxCui': 'str'}).set_index('RxCui'), on='RxCui')
    
    
    pro_df.reset_index(drop=True, inplace=True)
    med_df.reset_index(drop=True, inplace=True)
    
    int_df = pd.concat([pro_df, med_df], axis=1)
    intervention_num_df.append(int_df)



In [13]:
impression_num_df[0]

,ICD-10-CM,Frequency (Number of Calls),0,1,2
0,G89.2,85,Pain - Chronic,NaN,NaN
1,I10,227,Hypertension,CV - Hypertension,NaN
2,J80,891,Respiratory Distress - Acute Onset,Respiratory - Acute Onset Distress,NaN
3,E13.65,359,Hyperglycemia - Diabetic,Endocrine - Hyperglycemia - Diabetic,NaN
4,F10.92,286,Abuse of Alcohol - Intoxication,NaN,NaN
...,...,...,...,...,...
171,G04.90,4,Encephalitis or Encephalomyelitis,NaN,NaN
172,O75,5,OB - Childbirth Complicated,NaN,NaN
173,O14.9,2,OB - Pre-eclampsia,NaN,NaN
174,E34.9,2,Endocrine disorder - other,Endocrine Disorder - Otherwise Not Listed,NaN


In [14]:
intervention_num_df[0]

,Snomed,Proced. Frequency (Number of Calls),0,Unnamed: 2,Unnamed: 3,Unnamed: 4,RxCui,Med. Frequency (Number of Calls),Medications,Other Names,Other Names.1,Unnamed: 4
0,428803005,14767,CV - ECG - 4 Lead Monitoring,NaN,NaN,NaN,7806,3523.0,Oxygen,NaN,NaN,NaN
1,268400002,12010,CV - ECG - 12 Lead Obtained,12-Lead ECG,ECG,NaN,1191,1305.0,Aspirin (ASA),Aspirin,NaN,NaN
2,386423001,158,Behavior - Restraint Applied (Physical),Restraints,NaN,NaN,6960,383.0,Midazolam (Versed),NaN,NaN,NaN
3,392230005,9248,IV Start - Extremity Vein (arm or leg),NaN,NaN,NaN,125464,1605.0,Normal saline,Normal Saline,NaN,NaN
4,49689007,285,MS - Application of cervical collar,X-Collar-Only Immobilization,NaN,NaN,26225,501.0,Ondansetron (Zofran),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
89,226005007,1,CC - Central Line Maintenance / Care,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,2267008,1,Resp - Tracheostomy Tube Change,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,40174006,1,Infectious - Isolation Procedure Patient,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,396540005,1,IV - Blood Draw via IV or Venous Puncture,Blood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# now combine dictionaries

final_imp_dict = {}
final_pro_dict = {}
final_med_dict = {}

for i in range(len(years)):
    imp_dict = impression_num_dictionaries[i]
    pro_dict = pro_num_dictionaries[i]
    med_dict = med_num_dictionaries[i]
    

    for iD, freq in imp_dict.items():
        if iD not in final_imp_dict:
            final_imp_dict[iD] = 0
        final_imp_dict[iD] += freq
    
    for iD, freq in pro_dict.items():
        if iD not in final_pro_dict:
            final_pro_dict[iD] = 0
        final_pro_dict[iD] += freq
    
    for iD, freq in med_dict.items():
        if iD not in final_med_dict:
            final_med_dict[iD] = 0
        final_med_dict[iD] += freq


In [16]:
final_imp_dict

{'G89.2': 260,
 'I10': 639,
 'J80': 3060,
 'E13.65': 858,
 'F10.92': 923,
 'J98.9': 1864,
 'K08.8': 16,
 'S09.93': 369,
 'R10.84': 1624,
 'I20.9': 4012,
 'S09.90': 876,
 'R41.82': 5842,
 'F10.9': 373,
 'R11': 1449,
 'R11.1': 311,
 'G40.909': 1920,
 'R50.9': 770,
 'R55': 1495,
 'R51': 784,
 'T65.9': 479,
 'F11': 2607,
 'R53.1': 4113,
 'R53.81': 2313,
 'J96.9': 354,
 'R45.5': 150,
 'Z00': 1570,
 'I46.9': 1257,
 'J44.1': 2244,
 'S19.9': 282,
 'S39.92': 241,
 'A41.9': 729,
 'R07.89': 2817,
 'B99.9': 545,
 'G89.1': 2350,
 'M54.9': 739,
 'S69.9': 265,
 'K59.1': 363,
 'I49.9': 984,
 'E86.0': 422,
 'J45.901': 2112,
 'I20.0': 1589,
 'R10.0': 1003,
 'O03': 37,
 'J18.9': 220,
 'I50.9': 499,
 'J81.0': 316,
 'K59.00': 97,
 'J00': 205,
 'H81.399': 115,
 'J11': 355,
 'F41.9': 1764,
 'H57.10': 25,
 'F99': 377,
 'F14': 321,
 'G43.9': 125,
 'S09.92': 25,
 'R60.1': 293,
 'R46.2': 547,
 'S06.0X9A': 270,
 'S79.92': 214,
 'I26': 54,
 'I95.9': 740,
 'E13.64': 1097,
 'T14.91': 214,
 'K92.0': 155,
 'J98.01': 5

In [17]:
impression_num_dictionaries[0]

{'G89.2': 85,
 'I10': 227,
 'J80': 891,
 'E13.65': 359,
 'F10.92': 286,
 'J98.9': 775,
 'K08.8': 6,
 'S09.93': 148,
 'R10.84': 549,
 'I20.9': 1409,
 'S09.90': 428,
 'R41.82': 2460,
 'F10.9': 92,
 'R11': 782,
 'R11.1': 150,
 'G40.909': 784,
 'R50.9': 389,
 'R55': 642,
 'R51': 333,
 'T65.9': 231,
 'F11': 1143,
 'R53.1': 1530,
 'R53.81': 911,
 'J96.9': 182,
 'R45.5': 51,
 'Z00': 868,
 'I46.9': 532,
 'J44.1': 746,
 'S19.9': 113,
 'S39.92': 89,
 'A41.9': 337,
 'R07.89': 1277,
 'B99.9': 245,
 'G89.1': 1067,
 'M54.9': 310,
 'S69.9': 102,
 'K59.1': 142,
 'I49.9': 396,
 'E86.0': 163,
 'J45.901': 533,
 'I20.0': 503,
 'R10.0': 386,
 'O03': 16,
 'J18.9': 63,
 'I50.9': 207,
 'J81.0': 123,
 'K59.00': 33,
 'J00': 70,
 'H81.399': 47,
 'J11': 182,
 'F41.9': 641,
 'H57.10': 12,
 'F99': 186,
 'F14': 128,
 'G43.9': 43,
 'S09.92': 13,
 'R60.1': 155,
 'R46.2': 225,
 'S06.0X9A': 88,
 'S79.92': 98,
 'I26': 20,
 'I95.9': 320,
 'E13.64': 277,
 'T14.91': 84,
 'K92.0': 69,
 'J98.01': 115,
 'R44.1': 20,
 'S89.9': 

In [18]:
final_pro_dict

{'428803005': 30564,
 '268400002': 30491,
 '386423001': 416,
 '392230005': 26650,
 '49689007': 577,
 '353008': 229,
 '230040009': 634,
 '425543005': 1510,
 '425447009': 2124,
 '385857005': 60,
 '429283006': 435,
 '232674004': 324,
 '430824005': 527,
 '89666000': 932,
 '302789003': 247,
 '47545007': 833,
 '398041008': 312,
 '56251003': 951,
 '424287005': 14,
 '79321009': 242,
 '450611000124100': 323,
 '405427009': 352,
 '302488007': 10,
 '7443007': 329,
 '182692007': 1031,
 '26906007': 117,
 '406164000': 35,
 '426220008': 554,
 '422618004': 575,
 '236973005': 2,
 '51717002': 152,
 '243140006': 185,
 '449199004': 87,
 '182531007': 111,
 '233169004': 64,
 '235425002': 62,
 '426498007': 424,
 '229585002': 77,
 '409088002': 8,
 '425519007': 33,
 '271280005': 12,
 '447686008': 188,
 '20655006': 70,
 '19861002': 34,
 '128968000': 54,
 '182705007': 45,
 '18590009': 23,
 '46825001': 8061,
 '225116006': 14,
 '250980009': 29,
 '398074008': 9,
 '225358003': 51,
 '398279005': 8,
 '225150002': 39,
 

In [19]:
# convert dictionaries to dataframes 

final_imp_df = pd.DataFrame(list(final_imp_dict.items()),columns = ['ICD-10-CM','Frequency (Number of Calls)']) #convert
final_imp_df = final_imp_df.join(grouped_imp_df.set_index('ICD-10-CM'), on='ICD-10-CM')

pro_df = pd.DataFrame(list(final_pro_dict.items()),columns = ['Snomed', 'Proced. Frequency (Number of Calls)']) #convert
med_df = pd.DataFrame(list(final_med_dict.items()),columns = ['RxCui', 'Med. Frequency (Number of Calls)']) #convert


pro_df = pro_df.join(grouped_pro_df.astype({'Snomed': 'str'}).set_index('Snomed'), on='Snomed')
med_df = med_df.join(grouped_med_df.astype({'RxCui': 'str'}).set_index('RxCui'), on='RxCui')

pro_df.reset_index(drop=True, inplace=True)
med_df.reset_index(drop=True, inplace=True)
    
final_int_df = pd.concat([pro_df, med_df], axis=1)




In [20]:
#COMBINED - write out to excel in one sheet

writer = pd.ExcelWriter('COMBINED With ID - Frequencies of Interventions and Impressions RAA Data, 2017-2020.xlsx', engine='xlsxwriter')

final_imp_df.to_excel(writer, sheet_name='Impressions')
final_int_df.to_excel(writer, sheet_name='Interventions')

writer.save()

#by year - write out to excel in one sheet

imp_writer = pd.ExcelWriter('With ID by year - Frequencies of Impressions in RAA Data, ' + years[-1] + '-' + years[0] + '.xlsx', engine='xlsxwriter')
int_writer = pd.ExcelWriter('With ID by year - Frequencies of Interventions in RAA Data, ' + years[-1] +'-' + years[0] + '.xlsx', engine='xlsxwriter')
    
for i in range(len(years)):     
    impression_num_df[i].to_excel(imp_writer, sheet_name=str(years[i]))
    intervention_num_df[i].to_excel(int_writer, sheet_name=str(years[i]))

imp_writer.save()
int_writer.save()
    
